<a href="https://colab.research.google.com/github/RemyLpr/defi_ia/blob/main/data/req/Requetes_Test_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requêtage sur le test set

In [18]:
import pandas as pd
import numpy as np
import random
import string
import urllib.parse
import requests

## Chargement test set

In [2]:
df_test = pd.read_csv('test_set.csv')
df_test = df_test.drop(['index', 'avatar_id', 'hotel_id', 'stock'], axis=1)

# df avec 1 ligne par paramètres de requêtes
df_req = df_test.drop_duplicates(subset=['order_requests'], ignore_index=True)
df_req

,order_requests,city,date,language,mobile
0,1,vilnius,21,romanian,0
1,2,vilnius,18,romanian,0
2,3,paris,39,swedish,1
3,4,paris,36,swedish,1
4,5,paris,15,swedish,0
...,...,...,...,...,...
839,840,amsterdam,2,slovene,1
840,841,vienna,35,irish,1
841,842,vienna,6,irish,1
842,843,rome,5,irish,0


In [3]:
df_req[df_req['order_requests']==53]

,order_requests,city,date,language,mobile
52,53,madrid,0,german,1


## Moyen de requêter test set avec date -1 et date +1

### Définition des paramètres de requêtes

Faire une fonction qui parcourt le df req et prend les mêmes paramètres. Utiliser le numéro de ligne pour savoir quelle ligne utiliser dans les paramètres. (savoir où on en est). (pas utiliser la date)

In [32]:
def params_around_date(df_req, index_order_requests, date_b_a):
  # on fait -1 à l'index car par exemple si l'on veut les paramètres de order_requests=1, on veut l'index 0
  city = df_req.iloc[index_order_requests-1]['city']
  if date_b_a == 'b': #date avant
    date = df_req.iloc[index_order_requests-1]['date'] -1
  else: #date après
    date = df_req.iloc[index_order_requests-1]['date'] +1
  language = df_req.iloc[index_order_requests-1]['language']
  mobile = df_req.iloc[index_order_requests-1]['mobile']
  avatar = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
  params = {"city": city, "date": date, "language": language, "mobile": mobile, "avatar_name": avatar}
  
  return params

### Requêtage

In [10]:
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)
user_id = 'e68dad03-0d1a-4038-8766-382b25fe4252'

In [47]:
r = requests.get(path(f"remaining-requests/{user_id}"))
print("Nombre de requêtes restantes :", r.json())

Nombre de requêtes restantes : 223


In [46]:
nb_req = int(input("Entrez le nombre de requêtes à exécuter "))
index_order_requests = 54
old_df = pd.read_csv("train_set.csv")
df = old_df.copy()
req_count = int(old_df["order_requests"].tolist()[-1]) # on récupère la dernière colonne (order_requests) de la dernière ligne
i = 1
while (i < nb_req +1):
  for d in ['b', 'a']:
    params = params_around_date(df_req, index_order_requests=index_order_requests, date_b_a=d)
    if params['date']==-1:
      continue
    elif params['date']==45:
      continue
    else:
      requests.post(path(f'avatars/{user_id}/{params["avatar_name"]}')) # on doit indiquer à l'API qu'on crée ce nouvel avatar
      r = requests.get(path(f"pricing/{user_id}"), params = params)
      result = r.json() # résultat de la requête au format json
      query_res = pd.DataFrame(result['prices'])
      params_df = pd.DataFrame([result['request']]*len(query_res))
      new_df = pd.concat([query_res, params_df], axis=1)
      new_df["avatar_name"] = params["avatar_name"] # ajout du nom de l'avatar
      new_df["order_requests"] = req_count + i # ajout de la colonne order_requests comme dans le test_set
      # on ajoute les données extraites à notre dataset
      df = pd.concat([df, new_df])
      df.to_csv("train_set.csv", index=False) # pour pas écrire la colonne Unnamed : 0
      i += 1
  index_order_requests += 1
print("Requêtage terminé !")
print('prochain index order requests à mettre à la ligne 2 de cette cellule :', index_order_requests)

Entrez le nombre de requêtes à exécuter 100
Requêtage terminé !
prochain index order requests à mettre à la ligne 2 de cette cellule : 107
